In [3]:
# ============================================================
# PATIENTS: ANNUAL -> MONTHLY (données en mois et année)
# - Même logique que logistics-saisonnalite, sources patients
# - Lecture du CSV annuel patients, génère un CSV mensuel
# ============================================================

import pandas as pd
import numpy as np

# ----------------------------
# 1) Lecture du CSV annuel (patients)
# ----------------------------
CSV_PATH = "../data/patients/patients-data-interpolated.csv"
df_annual = pd.read_csv(CSV_PATH)

# ----------------------------
# 2) Paramètres mensuels (répartition normale)
# ----------------------------
MONTH_PCT = {
    1: 15,
    2: 10,
    3: 7,
    4: 6,
    5: 5,
    6: 6,
    7: 13,
    8: 13,
    9: 6,
    10: 5,
    11: 4,
    12: 10,
}
if sum(MONTH_PCT.values()) != 100:
    raise ValueError(f"Les pourcentages mensuels doivent faire 100. Total={sum(MONTH_PCT.values())}")

# ----------------------------
# 2a) Paramètres mensuels crise (COVID / tension)
# ----------------------------

MONTH_PCT_COVID = {
    1: 12,  # Tension hivernale
    2: 7,   # Baisse hivernale
    3: 15,  # Montée de vague
    4: 18,  # Pic printemps
    5: 8,   # Déclin
    6: 4,
    7: 3,
    8: 3,
    9: 6,   # Pré-vague automne
    10: 10, # Montée vague automne
    11: 9,  # Automne fort
    12: 5,  # Fin d’année
}

if sum(MONTH_PCT_COVID.values()) != 100:
    raise ValueError(f"MONTH_PCT_COVID doit faire 100. Total={sum(MONTH_PCT_COVID.values())}")

# ----------------------------
# 2b) Arrondi par unité (patients)
# ----------------------------
ROUNDING_BY_UNIT = {
    "patients": 0,
    "%": 2,
    "colis": 0,
    "repas": 0,
    "kg": 0,
    "t": 2,
    "km": 1,
    "m2": 1,
}
DEFAULT_DECIMALS = 2

# ----------------------------
# 3) Fonction annual -> daily
# ----------------------------
def annual_to_daily(
    annual_value: float,
    year: int,
    month_pct: dict,
    weekend_factor: float = 0.80,
    weekday_factor: float = 1.05,
    noise_sigma: float = 0.08,
    seed: int = 42
) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    dates = pd.date_range(f"{year}-01-01", f"{year}-12-31", freq="D")
    df = pd.DataFrame({"date": dates})
    df["year"] = year
    df["month"] = df["date"].dt.month
    df["dow"] = df["date"].dt.weekday  # 0=lundi ... 6=dimanche

    values = np.zeros(len(df), dtype=float)

    for m in range(1, 13):
        idx = df.index[df["month"] == m].to_numpy()
        monthly_total = float(annual_value) * (month_pct[m] / 100.0)
        base = np.where(df.loc[idx, "dow"].to_numpy() >= 5, weekend_factor, weekday_factor).astype(float)
        if noise_sigma > 0:
            noise = np.exp(rng.normal(loc=0.0, scale=noise_sigma, size=len(idx)))
        else:
            noise = np.ones(len(idx), dtype=float)
        weights = base * noise
        if weights.sum() == 0:
            weights = np.ones_like(weights)
        values[idx] = monthly_total * (weights / weights.sum())

    df["value"] = values
    df.loc[df.index[-1], "value"] += float(annual_value) - df["value"].sum()
    return df[["date", "year", "month", "dow", "value"]]

# ----------------------------
# 4) Vérification (mensuel)
# ----------------------------
def check_monthly_distribution(daily_df: pd.DataFrame, month_pct: dict) -> pd.DataFrame:
    monthly = daily_df.groupby("month")["value"].sum()
    pct_observed = (monthly / monthly.sum() * 100).round(2)
    out = pd.DataFrame({
        "pct_target": pd.Series(month_pct).sort_index(),
        "pct_observed": pct_observed
    })
    out["diff"] = (out["pct_observed"] - out["pct_target"]).round(2)
    return out

# ----------------------------
# 5) Génération journalier pour tout le CSV (PLF & CFX)
# ----------------------------
DEFAULT_PARAMS = {
    "weekend_factor": 0.80,
    "weekday_factor": 1.05,
    "noise_sigma": 0.08,
    "seed": 42,
}

all_daily = []

for i, row in df_annual.iterrows():
    year = int(row["ANNEE"])
    indic = row.get("INDICATEUR", "")
    sous_indic = row.get("SOUS-INDICATEUR", "")
    unite = row.get("UNITE", "")

    params = DEFAULT_PARAMS.copy()

    for site in ["PLF", "CFX"]:
        annual_value = row[site]

        if pd.isna(annual_value):
            continue

        df_d = annual_to_daily(
            annual_value=float(annual_value),
            year=year,
            month_pct=MONTH_PCT,
            **params
        )

        df_d["site_code"] = site
        df_d["indicateur"] = indic
        df_d["sous_indicateur"] = sous_indic
        df_d["unite"] = unite

        decimals = ROUNDING_BY_UNIT.get(str(unite).strip(), DEFAULT_DECIMALS)
        df_d["value"] = df_d["value"].round(decimals)

        all_daily.append(df_d)

df_daily_all = pd.concat(all_daily, ignore_index=True)

# ----------------------------
# 5b) Agrégation en données mensuelles (mois et année)
# ----------------------------
df_monthly_all = (
    df_daily_all
    .groupby(["year", "month", "site_code", "indicateur", "sous_indicateur", "unite"], as_index=False)
    .agg({"value": "sum"})
)
# Crise: répartition mensuelle selon MONTH_PCT_COVID (même total annuel, profil différent)
df_monthly_all["_annual"] = df_monthly_all.groupby(["year", "site_code", "indicateur", "sous_indicateur"])["value"].transform("sum")
df_monthly_all["value_crise"] = df_monthly_all["_annual"] * df_monthly_all["month"].map(lambda m: MONTH_PCT_COVID[m] / 100)
df_monthly_all = df_monthly_all.drop(columns=["_annual"])
# Réappliquer l'arrondi par unité (value et value_crise)
for unite_val, dec in ROUNDING_BY_UNIT.items():
    mask = df_monthly_all["unite"].astype(str).str.strip() == str(unite_val).strip()
    df_monthly_all.loc[mask, "value"] = df_monthly_all.loc[mask, "value"].round(dec)
    df_monthly_all.loc[mask, "value_crise"] = df_monthly_all.loc[mask, "value_crise"].round(dec)
other = ~df_monthly_all["unite"].astype(str).str.strip().isin([str(u).strip() for u in ROUNDING_BY_UNIT])
df_monthly_all.loc[other, "value"] = df_monthly_all.loc[other, "value"].round(DEFAULT_DECIMALS)
df_monthly_all.loc[other, "value_crise"] = df_monthly_all.loc[other, "value_crise"].round(DEFAULT_DECIMALS)

# ----------------------------
# 5c) Suppression des données vides (valeur totale = 0 pour même année/site/indicateur/sous-indicateur)
# ----------------------------
total_par_groupe = df_monthly_all.groupby(["year", "site_code", "indicateur", "sous_indicateur"])["value"].transform("sum")
df_monthly_all = df_monthly_all[total_par_groupe > 0].copy()

# ----------------------------
# 6) Exemple de contrôle sur un cas
# ----------------------------
example = df_monthly_all.iloc[0:0].copy()
if len(df_monthly_all) > 0:
    first = df_monthly_all.iloc[0]
    example = df_monthly_all[
        (df_monthly_all["year"] == first["year"]) &
        (df_monthly_all["site_code"] == first["site_code"]) &
        (df_monthly_all["indicateur"] == first["indicateur"]) &
        (df_monthly_all["sous_indicateur"] == first["sous_indicateur"])
    ]
    print("---- Vérification % mensuels (exemple) ----")
    pct_obs = (example.groupby("month")["value"].sum() / example["value"].sum() * 100).round(2)
    print(pd.DataFrame({"pct_target": pd.Series(MONTH_PCT).sort_index(), "pct_observed": pct_obs}))

print("\nSomme totale (toutes lignes) =", df_monthly_all["value"].sum())

# Répartition par année / mois / indicateur / sous-indicateur
summary = (
    df_monthly_all
    .groupby(["year", "month", "indicateur", "sous_indicateur"])["value"]
    .sum()
    .reset_index()
    .sort_values(["year", "month", "indicateur", "sous_indicateur"])
)
print("\nRépartition mensuelle par année / indicateur / sous-indicateur :")
print(summary)

# ----------------------------
# 7) Export final (données en mois et année)
# ----------------------------
OUT_PATH = "../data/patients/patients-donnees_mensuelles_reconstituees.csv"
df_monthly_all.to_csv(OUT_PATH, index=False, encoding="utf-8")
print(f"\n✅ Fichier exporté: {OUT_PATH}")

# Aperçu (10 premières lignes)
df_monthly_all.head(10)

---- Vérification % mensuels (exemple) ----
    pct_target  pct_observed
1           15         14.96
2           10          9.98
3            7          7.00
4            6          6.02
5            5          5.03
6            6          5.99
7           13         12.94
8           13         13.02
9            6          6.04
10           5          5.01
11           4          3.94
12          10         10.06

Somme totale (toutes lignes) = 561526.6300000001

Répartition mensuelle par année / indicateur / sous-indicateur :
      year  month                indicateur  \
0     2011      1  Causes d'hopitalisations   
1     2011      1  Causes d'hopitalisations   
2     2011      1      Origine géographique   
3     2011      1      Origine géographique   
4     2011      1      Origine géographique   
...    ...    ...                       ...   
1687  2016     12      Origine géographique   
1688  2016     12           Profil patients   
1689  2016     12           Profil patie

,year,month,site_code,indicateur,sous_indicateur,unite,value,value_crise
0,2011,1,CFX,Causes d'hopitalisations,Pathologies cancéreuses - Nouveaux,patients,592.00,475.00
1,2011,1,CFX,Causes d'hopitalisations,Pathologies cancéreuses - Total,patients,833.00,665.00
2,2011,1,CFX,Origine géographique,Autres départements et DOM TOM,%,0.31,0.15
5,2011,1,CFX,Origine géographique,Province - MCO,%,0.85,0.69
6,2011,1,CFX,Origine géographique,Province - SSR,%,0.31,0.17
7,2011,1,CFX,Origine géographique,Val-de-Marne - MCO,%,1.10,0.86
8,2011,1,CFX,Origine géographique,Val-de-Marne - SSR,%,2.30,1.83
11,2011,1,CFX,Origine géographique,Île-de-France - Hauts-de-Seine - MCO,%,0.56,0.49
12,2011,1,CFX,Origine géographique,Île-de-France - Autres - MCO,%,1.14,0.90
13,2011,1,CFX,Origine géographique,Île-de-France - Autres - SSR,%,0.48,0.37


In [4]:
# Répartition par site et indicateur (données mensuelles)
df_monthly_all["indicateur"].value_counts()
df_monthly_all.groupby(["site_code", "indicateur"]).size()

site_code  indicateur              
CFX        Causes d'hopitalisations     144
           Origine géographique        1152
           Profil patients              324
           Urgences                      72
PLF        Causes d'hopitalisations     144
           Origine géographique        1152
           Profil patients              324
           Urgences                      72
dtype: int64

In [3]:
# ============================
# Graph: Patients (mensuel)
# - Charge le CSV mensuel reconstitué
# - Filtre indicateur + sous-indicateur (ex: Causes d'hopitalisations / Pathologies cancéreuses - Nouveaux)
# - Choisit un site (PLF/CFX) et une année
# - Trace la courbe par mois (+ optionnel: moyenne mobile 3 mois)
# ============================

import pandas as pd
import matplotlib.pyplot as plt

# 1) Chemin vers le fichier mensuel généré
MONTHLY_CSV_PATH = "../data/patients/patients-donnees_mensuelles_reconstituees.csv"

df = pd.read_csv(MONTHLY_CSV_PATH)

# 2) Date pour l'axe x (1er du mois)
df["date"] = pd.to_datetime(df[["year", "month"]].assign(day=1))

# 3) Paramètres du graphe
SITE = "PLF"         # "PLF" ou "CFX"
YEAR = 2012          # année à visualiser
INDICATEUR = "Causes d'hopitalisations"
SOUS_INDICATEUR = "Pathologies cancéreuses - Nouveaux"  # adapter selon les données

# 4) Filtre
mask = (
    (df["site_code"] == SITE) &
    (df["year"] == YEAR) &
    (df["indicateur"].str.strip() == INDICATEUR) &
    (df["sous_indicateur"].str.strip() == SOUS_INDICATEUR)
)

data_plot = df.loc[mask, ["date", "year", "month", "value", "unite"]].sort_values("date").copy()

if data_plot.empty:
    print("❌ Aucune donnée trouvée avec ces filtres.")
    print("➡️ Vérifie les valeurs avec : df[['indicateur','sous_indicateur']].drop_duplicates()")
else:
    unit = data_plot["unite"].iloc[0] if "unite" in data_plot.columns and len(data_plot) > 0 else ""
    data_plot["mm3"] = data_plot["value"].rolling(3, min_periods=1).mean()

    # 5) Plot (par mois)
    plt.figure(figsize=(14, 5))
    plt.plot(data_plot["date"], data_plot["value"], "o-", label="Mensuel")
    plt.plot(data_plot["date"], data_plot["mm3"], label="Moyenne mobile 3 mois")
    plt.title(f"{INDICATEUR} - {SOUS_INDICATEUR} ({SITE}) - {YEAR} (par mois)")
    plt.xlabel("Mois")
    plt.ylabel(f"Volume ({unit})" if unit else "Volume")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    display(data_plot.head(12))

❌ Aucune donnée trouvée pour Déchets / Cartons avec ces filtres.
➡️ Vérifie les valeurs exactes de 'indicateur' et 'sous_indicateur' avec :
   df[df["indicateur"]=="Déchets"]["sous_indicateur"].unique()
